In [4]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import holidays

# 1. 날짜 범위 설정
start_date = datetime(2018, 1, 1)
end_date = datetime.today()
date_range = pd.date_range(start=start_date, end=end_date)

# 2. 한국 공휴일 정보 가져오기
kr_holidays = holidays.KR(years=range(2018, end_date.year + 1))

# 3. 설날 / 추석 기준일 리스트
holiday_dates = [
    # 설날
    datetime(2018, 2, 16), datetime(2019, 2, 5), datetime(2020, 1, 25),
    datetime(2021, 2, 12), datetime(2022, 2, 1), datetime(2023, 1, 22),
    datetime(2024, 2, 10), datetime(2025, 1, 29),
    # 추석
    datetime(2018, 9, 24), datetime(2019, 9, 13), datetime(2020, 10, 1),
    datetime(2021, 9, 21), datetime(2022, 9, 10), datetime(2023, 9, 29),
    datetime(2024, 9, 17), datetime(2025, 10, 6),
]

# 4. 명절 영향 지수 계산 함수 / 명절 이전기간이 더 크게 영향을 주고 양상이 다르므로
def compute_holiday_score_asymmetric(date, holidays, pre_window=14, post_window=7):
    score = 0.0
    for h in holidays:
        diff = (date - h).days

        if -pre_window <= diff < 0:
            # 명절 전: 완만한 선형 감소
            score = max(score, round(1 - (abs(diff) / pre_window), 4))
        elif diff == 0:
            score = max(score, 1.0)
        elif 0 < diff <= post_window:
            # 명절 후: 더 빠른 감쇠
            score = max(score, round(0.7 - (diff / post_window) * 0.7, 4))
    return score


# 5. 데이터프레임 생성
df = pd.DataFrame({'date': date_range})
df['weekday'] = df['date'].dt.weekday  # 0=월, 6=일
df['is_weekend'] = df['weekday'].isin([5, 6])  # 토/일 여부
df['is_kor_holiday'] = df['date'].isin(kr_holidays)
df['holiday_flag'] = (df['is_weekend'] | df['is_kor_holiday']).astype(int)

df['holiday_score'] = df['date'].apply(lambda x: compute_holiday_score_asymmetric(x, holiday_dates))

# 6. 정리 및 저장
df_final = df[['date', 'holiday_flag', 'holiday_score']]
df_final.to_csv('holiday_score_table.csv', index=False, encoding='cp949')